In [ ]:
import arcpy
from dataclasses import dataclass
from archaic import FeatureClass

arcpy.env.workspace = ".data/canada.geodatabase"  # type: ignore


@dataclass(frozen=True)
class Mine:
    oid: int
    name_e: str
    type_e: str
    shape: arcpy.Multipoint


feature_class = FeatureClass[Mine]("main.mines_pt")

for mine in feature_class.read([1, 2, 7]):
    print(mine)

feature_class.info.__dict__